# Laboratorio #3 - Reconocimiento de imagenes mediante CNN
## Álvaro Andrés Esquivel Gómez 11002822

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras

## Arquitectura de la red convolucional

Para esta red convolucional se utilizó una arquitectura piramidal con 3 capas convolucionales y 3 operaciones de MaxPolling, según la estructura de la red piramidal, para la primera capa se configuro con 32 filtros, para la secunda con el doble (64) y para la tercera con 128 filtros. La idea es que a medida que la imagen se propaga por las capas, cada una de ellas tiene más capacidad para la detección de patrones que la identifiquen y se va haciendo más pequeña.

Para la capa Fully-connected se diseñó con 128 neuronas para la primer capa densa y 64 neuronas para la segunda capa, por último, la salida es igual al ejercicio con una neurona con activación sigmoid (la única de este tipo, todas las demás son relu).


In [2]:
#Modelo de Red Convolucional

#Estructura base
cnn = Sequential()

#Primera capa convolucional
cnn.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64,64,3), activation="relu"))

#Maxpooling
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,2)))

#Capa de Flattening
cnn.add(Flatten())

#Fully-connected
cnn.add(Dense(units=128, activation="relu"))
cnn.add(Dropout(0.5))

#Fully-connected
cnn.add(Dense(units=64, activation="relu"))
cnn.add(Dropout(0.5))

#Capa de salida para clasificacion binaria
cnn.add(Dense(units=1, activation="sigmoid"))

In [3]:
#Compilar la red
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
strDataTrain='dataset/training_set'
strDataTest='dataset/test_set'

In [5]:
#Preprocesamiento para train con data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255., ##Rescala la imagen a nuemero entre 0 y 1
    shear_range=0.2, ##Torcer un poco la imagen  
    zoom_range=0.2,
    horizontal_flip=True
)

#Preprocesamiento para test
test_datagen = ImageDataGenerator(rescale=1./255.)

#Configuracion de imagenes de entrada para train
train_set = train_datagen.flow_from_directory(
    strDataTrain,
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

#Configuracion de imagenes de entrada para test
test_set = test_datagen.flow_from_directory(
    strDataTest,
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Callbacks 

Se agregan callbacks en el modelo, el primero con la finalidad de almacenar el mejor modelo, debido al alto tiempo de entrenamiento es posible que ocurra algún fallo, este callback almacena siempre el mejor modelo entrenado hasta el momento para no perderlo.

El segundo es de tipo Early Stopping con la finalidad de identificar cuando el entrenamiento no encuentre mejores resultados se pueda detener despues de 10 epochs, con la finalidad de prevenir la divergencia.


In [8]:
#Callbacks
checkpoint_cb = ModelCheckpoint("modelo_lab3.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

In [9]:
#Entrenamiento del modelo
cnn.fit(train_set, 
        steps_per_epoch=8000,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000,
        verbose=1,
       callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/25
8000/8000 [==============================] - 2320s 290ms/step - loss: 0.4106 - accuracy: 0.8032 - val_loss: 0.6255 - val_accuracy: 0.8433
Epoch 2/25
8000/8000 [==============================] - 1908s 238ms/step - loss: 0.2181 - accuracy: 0.9107 - val_loss: 0.3026 - val_accuracy: 0.8522
Epoch 3/25
8000/8000 [==============================] - 1900s 238ms/step - loss: 0.1502 - accuracy: 0.9414 - val_loss: 0.3064 - val_accuracy: 0.8651
Epoch 4/25
8000/8000 [==============================] - 1924s 241ms/step - loss: 0.1119 - accuracy: 0.9573 - val_loss: 0.1505 - val_accuracy: 0.8616
Epoch 5/25
8000/8000 [==============================] - 1954s 244ms/step - loss: 0.0906 - accuracy: 0.9660 - val_loss: 0.4007 - val_accuracy: 0.8637
Epoch 6/25
8000/8000 [==============================] - 1917s 240ms/step - loss: 0.0775 - accuracy: 0.9717 - val_loss: 2.4867 - val_accuracy: 0.8549
Epoch 7/25
8000/8000 [==============================] - 1933s 242ms/step - loss: 0.0651 - accuracy: 0.9765

KeyboardInterrupt: 

El modelo entrenado tuvo una falla por el equipo alrededor de 7 horas después de empezar el entrenamiento, por suerte se puede recuperar el mejor modelo anterior a eso y predecir con ese.


In [12]:
#Importando el mejor modelo
cnn = keras.models.load_model("modelo_lab3.h5")

In [23]:
print("Metricas del modelo: ", cnn.metrics_names)

Metricas del modelo:  ['loss', 'accuracy']


In [24]:
#Evaluando el modelo
resultado_evaluacion = cnn.evaluate(test_set)

63/63 [==============================] - 7s 114ms/step


In [28]:
print("Accuracy del modelo: ", round(resultado_evaluacion[1], 2)*100, "%")

Accuracy del modelo:  87.0 %


## Resultados del modelo

Al evaluar el modelo se obtuvo un accuracy de 87% utilizando la arquitectura anteriormente indicada, y completando 16 epoch de entrenamiento, por lo que el resultado es muy bueno y efectivamente se cumple con lo requisitos solicitados.
